# Regular Expressions

Look at the file `data/nasa_web.log`. It contains two month's worth of all HTTP requests to the NASA Kennedy Space Center server in Florida. 
We are going to go though that file and extract some interesting information. 

Let's start by reading the file into your python session.

In [ ]:
with open('data/nasa_web.log', 'r') as f:
    log_file = f.read()

In [ ]:
log_file

If you're not familiar with HTTP logs, the data provided can be broken down like this:

`netspace.net.au - - [01/Jul/1995:03:34:44 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985`



| Host making the request | Timestamp            | Timezone      | HTTP method | URL                 | HTTP Version |HTTP status code | Bytes in the reply |
|-------------------------|----------------------|---------------|-------------|---------------------|-------------|-----------------|--------------------|
| netspace.net.au         | 01/Jul/1995:03:34:44 | -0400         | GET         | /shuttle/countdown/ | HTTP/1.0    | 200              | 3985               |

## The vocabulary

Regular expressions are a way to define strings that match a specific pattern. There is a whole language to be learned here and below you will find some of the most useful values.

    \d      Any Digit
    \D      Any Non-digit character
    .       Any Character
    \.      Period
    [abc]   Only a, b, or c
    [^abc]  Not a, b, nor c
    [a-z]   Characters a to z
    [0-9]   Numbers 0 to 9
    \w      Any Alphanumeric character
    \W      Any Non-alphanumeric character
    {m}     m Repetitions
    {m,n}   m to n Repetitions
    *       Zero or more repetitions
    +       One or more repetitions
    ?       Optional character
    \s      Any Whitespace
    \S      Any Non-whitespace character
    ^…$     Starts and ends
    (ab|de) Matches ab or de

[source](https://regexone.com/references/python)

A fantastic way of learning/practicing regex is [regex101](https://regex101.com/)

Firstly, try to understand the regular expression below:

In [ ]:
r'GET /\w+'

This expression would look for the word `GET` followed by a space character ` `, followed by a forward slash `/` and one or more alphanumeric character(s). In that sense this would be an expression to search for URLs requested with a GET.

*Although not always required, if you want to you can always define the strings with the `r''` prefix to make sure the are never interpreted, i.e. problems such Python using `\` as an escape character are avoided.*

### Exercise - match dates

Another example of a simple regular expression would be to search for a date. Imagine we wanted to capture all date instances that followed the convention 01/Jul/1995. What would an expression catch this pattern look like?

In [ ]:
# Add your code below
# write a regular expression pattern that can recognise dates in the log (you will not need to run it - discuss)



With our log we know exactly the number of letters and numbers expected. With regex, you can use `{n}` after a pattern to specify that you expect **n** instances.
For example `\d{2}` will match exactly 2 digits.

## `search()`

Python supports regular expressions through a built-in library called `re`. Start by importing the library and using 
the `.search()` function, which takes a regex pattern and a string to be searched. 

Search `log_file` for `GET` requests (the regex specified under the vocublary list above can be used), assigning the `re.Match object` (which is returned by `re.search()`, to the variable `match`:

In [ ]:
# Add your code below
import re
pattern = r'GET /\w+'



Note that the search function will only return the first match, so it is mostly useful to see whether your regular expression does at least have one hit. If there is no hit, it simply returns `None`. If it does find a hit, it returns the hit as a `MatchObject`.

We can use the `.group()` method of the `MatchObject` to extract the actual string that was matched:

In [ ]:
match.group()

We can use the `.start()` method of the `MatchObject` to extract the starting position of the string that was matched:

In [ ]:
match.start()

We can use the `.end()` method of the `MatchObject` to extract the end position of the string that was matched:

In [ ]:
match.end()

### Creating a message from `MatchObject` properties

Similarly to the `.group()` method which extracts the actual string which was matched, you can use the `.start()` and `.end()` methods to collect the position of where the regular expression matched.

`print` the `start` and `end` positions formatted nicely with the actual match. Use an f-string to put together the `.group()`, `.start()`, `.end()` into a sentence as a single string, assigned to `message`:

In [ ]:
# return a single string containing .group(), .start() and .end() of the match
message = f'Found "{match.group()}" at position: {match.start()}-{match.end()}'
message


## `findall()`

Search `log_file` for all matches of our regex `pattern`, we can use the `re.findall()` function. It takes the same parameters as `re.search()`, but returns a list of matches. 

Assigning the returned list to the variable `matches`:

In [ ]:
# Add your code below
pattern = r'GET /\w+'



## Substitutions

Regular expressions are useful in another scenario where we want to find and replace text. This can be useful when you want to change the domain name of an email of your business for instance.

Setting up a substitution is very similar to using regular expressions in general, except that this time we also need to specify a `replacement` value. 

Replace the word **'company'** in the email addresses found in **'contacts'** with the word **'agency'** using `sub()` method.

In [ ]:
# Add your code below
contacts = 'Please write to tom@company.com or margaret@company.com'
pattern = r'company'
replacement = r'agency'



### Exercise - Replace IP addresses

Now we want to **find** and **replace** all host **IP addresses** with the string **'unknown domain'** in `log_file`

Save the regular expression pattern string to a variable called `pattern`

In [ ]:
# Add your code below
# Step1 - develop the regular expression pattern string to find all IP addresses



In [ ]:
# If the above pattern is correct, you should see all IP addresses in 'log_file' by running the code below
re.findall(pattern, log_file)

Finally save the `log_file` to a new file called `data/log_file_edited.log` where every **IP addresses** is replaced with `unknown domain`.

In [ ]:
# Step2 - write substituted data to a file

replacement = r'unknown domain'

with open('data/log_file_edited.log', 'w') as f:
    f.write(re.sub(pattern, replacement, log_file))


## Additional examples

###### How many requests for a page from the `shuttle` section (base `/shuttle/`) failed (status code different than 200)?

In [ ]:
pattern = r'GET /shuttle/[\w/\.]* HTTP/1.0" (?!200)\d{3}'
# pattern = r'GET /shuttle/[\w/\.]* HTTP/1.0" ([0-13-9]\d{2}|2[1-9]\d|2\d[1-9])'
len(re.findall(pattern, log_file))


Hint: for the "different than 200" part you have two options; either use `pattern1|pattern2` in order to list all the accepted status codes, or read about negative look ahead `(?!pattern)` that allow you to match the string only if the pattern isn't found.

###### Get all different second level arguments to URL starting from shuttle (`/shuttle/something`)?

In [ ]:
pattern = r'GET /shuttle/(\w+)/'
set(re.findall(pattern, log_file))


###### How many queries did NASA get from a german domain (`.de`)?

In [ ]:
pattern = r'^[\w\.]+\.de'
len(re.findall(pattern, log_file, re.MULTILINE))


Hint: In order to match the beginning of a line use `^` and add `re.MULTILINE` as argument to `re.findall` (or it will be treated as the `not` operator)
For instance `re.findall(r'^a', file_, re.MULTILINE)` will find all lines starting with an `a`